In [ ]:
from comet_ml import Experiment
import numpy as np 
from matplotlib import pyplot as plt
import os 
import time 
import tensorflow as tf 


#ISIC 2019 25000 IMAGES DATASET LOCATION
train_path = "/home/mrb685/Datasets/Thesis Final Dataset/train/"
val_path  = "/home/mrb685/Datasets/Thesis Final Dataset/val/"
test_path = "/home/mrb685/Datasets/Thesis Final Dataset/test/"

CATEGORIES = ["NV", "MEL", "BCC", "BKL", "AK", "SCC", "VASC", "DF"]



# name used to save  
model_name = "all_base_"



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


### CALLBACKS 

# tensorboard callback 
tensorboard_name = f"Multimodal_skin_lesion_{model_name}"
tensorboard =  tf.keras.callbacks.TensorBoard(log_dir=f'logs/{tensorboard_name}')


# model checkpoint callback 
checkpoint_path = f"/home/mrb685/CheckPoints/{model_name}/cp.ckpt" 
checkpoint_dir = os.path.dirname(checkpoint_path)

if not os.path.isdir(checkpoint_dir):
    os.mkdir(checkpoint_dir)

if not os.path.isfile(checkpoint_path):
    open(checkpoint_path, 'w').close()
        


# Create a callback that saves the model's weights
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                mode='max')

lr_on_plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, min_lr=0.00001)

early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=10)

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

<h1>Preprocess Data <h1>

In [ ]:
import numpy as np
import sklearn.metrics as sklm

"""
    Define early stop behaviour
"""
class CustomMetricsAndEarlyStop(tf.keras.callbacks.Callback):
    
    def __init__(self, patience=5):
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
    
    def on_train_begin(self, logs={}):

        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = 0

    def on_epoch_end(self, epoch, logs={}):
        print(f"Log: {logs}")
        val_acc = logs['val_accuracy']

        
        current = val_acc
        if np.greater(current, self.best):
            save_path = f"/home/mrb685/Saved Models/{model_name}.h5"
            print(f"val_accuracy improved from {self.best} to {current}. Saving model!")
            self.model.save(save_path)
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('Restoring model weights from the end of the best epoch.')
                self.model.set_weights(self.best_weights)
        
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [ ]:
import threading
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self): # Py3
        with self.lock:
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g


"""
    Random crop large images 
"""
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

@threadsafe_generator
def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [ ]:
X, y = next(train_crop_generator)
for x in range(10):
    plt.imshow(X[x])
    plt.show()

<h1>Train Models<h1>

In [ ]:
# CREATE MODEL 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import VGG19, ResNet50, ResNet101V2, ResNet152, DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7 
from tensorflow.keras.models import Model
from sklearn.utils import class_weight
import numpy as np

# "VGG19", "ResNet50", "ResNet152", "DenseNet121", "DenseNet201", "efficientnet_b0", 
#               "efficientnet_b1", "efficientnet_b2", "efficientnet_b3", "efficientnet_b4"
# "efficientnet_b5", "efficientnet_b6"

# (16, 256, 224), (16, 256, 224), (16, 256, 224), (16, 256, 224), (8, 256, 224), 
#                  (8, 256, 224), (8, 256, 240), (8, 300, 260), (8, 350, 300), (8, 400, 380)

base_names = ["efficientnet_b0"]
datagen_param = [(4, 400, 224)]
for i in range(len(base_names)):
    
    BATCH_SIZE, IMG_SIZE, IMG_CROP_SIZE = datagen_param[i]
    
    name = base_names[i]
    if name == "VGG19":
        base = VGG19(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "ResNet50":
        base = ResNet50(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "ResNet101V2":
        base = ResNet101V2(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "ResNet152":
        base = ResNet152(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "DenseNet121":
        base = DenseNet121(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "DenseNet169":
        base = DenseNet169(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "DenseNet201":
        base = DenseNet201(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b0":
        base = EfficientNetB0(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b1":
        base = EfficientNetB1(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b2":
        base = EfficientNetB2(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b3":
        base = EfficientNetB3(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b4":
        base = EfficientNetB4(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b5":
        base = EfficientNetB5(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b6":
        base = EfficientNetB6(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    if name == "efficientnet_b7":
        base = EfficientNetB7(input_shape=(IMG_CROP_SIZE,IMG_CROP_SIZE,3),weights="imagenet",include_top=False)
    
    
    model_name = f"base_isic_randcrop_{name}"
    print("---------------------------------")
    print(name)
    print("---------------------------------")
    
    np.random.seed(1234)
    x = base.output
    x = Flatten()(x) 
    predictions = Dense(8, activation='softmax',name='output_layer')(x)
    
    model = Model(base.inputs, predictions)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['accuracy'])
    
    
    dot_img_file = f'model_arch_{name}.png'
    tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)
    
    continue 
    nLayers = len(model.layers)
#     print(nLayers)
    
    freeze_layers = int(nLayers * 0.20)
    
    for l in range(freeze_layers):
        model.layers[l].trainable = False
        
        
#     for layer in model.layers: 
#         print(layer.trainable)
    
    BATCH_SIZE, IMG_SIZE, IMG_CROP_SIZE = datagen_param[i]
    
    print(f"Datagen Param: {BATCH_SIZE}, {IMG_SIZE}, {IMG_CROP_SIZE}")
    
    datagen = ImageDataGenerator(rescale=1./255.,
                                 rotation_range=90,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 brightness_range=(0.8, 1.2),
                                validation_split=0.2)

    # datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)
    # val_datagen = ImageDataGenerator(rescale=1./255.)
    
    train_generator = datagen.flow_from_directory(
        target_size=(IMG_SIZE, IMG_SIZE),
        directory=train_path_isic,
        batch_size=BATCH_SIZE, 
        shuffle = True,
        classes=CATEGORIES,
        subset="training",
        class_mode="categorical")

    val_generator = datagen.flow_from_directory(
        batch_size=BATCH_SIZE,
        target_size=(IMG_SIZE, IMG_SIZE),
        directory = train_path_isic,
        shuffle = True,
        subset="validation",
        classes=CATEGORIES,
        class_mode = "categorical")

    train_crop_generator = crop_generator(train_generator, IMG_CROP_SIZE)
    val_crop_generator = crop_generator(val_generator, IMG_CROP_SIZE)
    
    # Run model 
    
    metrics = CustomMetricsAndEarlyStop(10)

    # Add the following code anywhere in your machine learning file
    # experiment = Experiment(api_key="ufQQBm4PhfVWxcwobrFapECw8",
    #                         project_name="skin-lesion-classification", workspace="rsd96")
    import time
    import multiprocessing
    t=time.time()
    
    # import comet_ml in the top of your file
    from comet_ml import Experiment
    
#     Add the following code anywhere in your machine learning file
    experiment = Experiment(api_key="ufQQBm4PhfVWxcwobrFapECw8",
                        project_name="skin-lesion-classification", workspace="rsd96")
    
    experiment.set_name(model_name)
    print(train_generator.samples)
    hist = model.fit(train_crop_generator,
                               validation_data = val_crop_generator,
                               epochs=100,
                               shuffle=True,
                               workers = multiprocessing.cpu_count(),
                               callbacks = [lr_on_plateau_callback, metrics],
                               steps_per_epoch=train_generator.samples // BATCH_SIZE,
                               validation_steps=val_generator.samples // BATCH_SIZE)

    experiment.end()
    print('Training time: %s' % (time.time() - t))
    
    

In [ ]:
# Plot and save model 

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model sensitivity')
plt.ylabel('sensitivity')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# model.save(f"{model_name}.model");

<h1>Test Trained Models<h1>

In [ ]:
# Create test data generator 
from keras_preprocessing.image import ImageDataGenerator

BATCH_SIZE = 16
datagen = ImageDataGenerator(rescale=1./255.)

test_generator = datagen.flow_from_directory(
    target_size=(IMG_SIZE, IMG_SIZE),
    directory=test_path,
    batch_size=BATCH_SIZE, 
    shuffle = True,
    classes=CATEGORIES,
    class_mode="categorical")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, auc, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.python.keras.utils.data_utils import Sequence

In [ ]:

model_names = ["VGG19", "ResNet50", "ResNet152", "DenseNet121", "DenseNet201",
               "efficientnet_b0", "efficientnet_b1", "efficientnet_b2", "efficientnet_b3", "efficientnet_b4"]

# model_names = ["efficientnet_b4"]
for name in model_names: 
    print("---------------------------------")
    print(name)
    print("---------------------------------")
    
#     layers = ["25", "50", "75"]
#     for layer in layers:
    model_name = f"base_isic_randcrop_{name}" 
    model = load_model(f"/home/mrb685/Saved Models/{model_name}.h5")


    num_of_test_samples = 1651

    #Confution Matrix and Classification Report
    Y_pred = model.predict_generator(val_generator, num_of_test_samples // BATCH_SIZE +1, verbose=1)
    y_pred = np.argmax(Y_pred, axis=1)
    target_names = ["NV", "MEL", "BCC", "BKL", "AK", "SCC", "VASC", "DF"]

    print('Confusion Matrix')
    cm = confusion_matrix(val_generator.classes, y_pred, normalize='true')

    ax= plt.subplot()

    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted');
    ax.set_ylabel('True'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(target_names); 
    ax.yaxis.set_ticklabels(target_names);
    plt.show()

    plt.savefig(f'/home/mrb685/Confusions/{name}.png')
    print('Classification Report')
    report = classification_report(val_generator.classes, y_pred, target_names=target_names, output_dict=True)
    df = pd.DataFrame(report).transpose()

    print(classification_report(val_generator.classes, y_pred, target_names=target_names))

    fpr, tpr, thresholds = roc_curve(val_generator.classes, y_pred, pos_label=2)
    print(f"AUC : {auc(fpr, tpr)}")